In [ ]:
import numpy as np
import os,inspect
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
#Capas
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Flatten
from keras.layers import Dropout
#Optimizadores
from keras.optimizers import Adam, RMSprop, SGD
#utilidades
from keras.utils import plot_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tqdm import tqdm
#utilitis
from keras.utils import plot_model
from keras.layers.normalization import BatchNormalization

from preprocesamiento import data
from keras.models import load_model, Model
import keras.backend as K
%matplotlib inline

In [ ]:
#datos historicos
n_in=4
#datos predecidos
n_out=1
#porcentage de datos para el entrenamiento
n_train=85
(x_train, y_train, x_val, y_val),var = data('union.xlsx', n_in, n_out, n_train,  "a")
n_in=4*var
n_out=n_out*var

In [ ]:
x_train=x_train.reshape((-1,x_train.shape[2], x_train.shape[1]))
x_val=x_val.reshape((-1,x_val.shape[2], x_val.shape[1]))

In [ ]:
x_train.shape

In [ ]:
x_train.min()

In [ ]:
def model(step_input=1,
              step_out=n_out,
              size_input=n_in,
              output_dim=1,
              encoder_units=64):
    '''
            step_input: longitud de la entrada hacia el lstm(datos pasados para predecir un valor futuro)
              step_out: alcanze de la inferencia, en nuestro caso se infiere un año,
              size_input: tamaño de la entrada de los datos,
              output_dim: tamaño de la salida de los datos,
              encoder_units: espacio dimencional de salida de la LSTM
    '''
    X = Input(shape = (step_input, size_input))
    #primera capa de LSTM
    #lstm= LSTM(encoder_units, activation="tanh", return_sequences=False)(X)
    #lstm= LSTM(encoder_units, activation="tanh", return_sequences=False)(lstm)
    #capa fully conected con una neurona
    out = Dense(64, activation='tanh')(X)
    out = Flatten()(out)
    out = Dense(12, activation='tanh')(out)
    model = Model(inputs = [X], outputs = out)
    ### END CODE HERE ###
        
    return model

In [ ]:
model = model()

In [ ]:
# summary model
model.summary()

In [ ]:
# plot model seq to seq (LSTM) with attention and save img in root directory
plot_model(model, to_file='modeloRNN.png', show_shapes=True)

# Entrenamiento del modelo

In [ ]:
# compile model 
#se utiliza el optimizador root mean square con los siguientes hyperparametros
#Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#RMSprop(lr=0.45, rho=0.94, epsilon=1.0, decay=0.1)
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#se utiliza la tecnica de early stoping con el objetivo de evitar el sobre ajuste de la red, y deteniendo
#el proceso se detiene despues de 4 itereaciones si no existiera una mejora
early_stopping=EarlyStopping(monitor='val_loss', patience=4)
#se guarda los pesos del modelo como pesos.h5 si en cada paso de todo el conjunto de datos existiera una mejora
mcp_save = ModelCheckpoint('pesos.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')
#las metricas de entrenamiento y la perdida se configuran con mean square error
model.compile(optimizer=opt, metrics=["mae"], loss='mae')

In [ ]:
# train model
History=model.fit(x_train, y_train, epochs=1000,  callbacks=[early_stopping,mcp_save], batch_size=4, validation_data=(x_val,y_val))

# Graficas

In [ ]:
#imprime las variables de medicion que serviran para graficar 
print(History.history.keys())

In [ ]:
# plot train and validation loss
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper right')
plt.show()

In [ ]:
#Se realiza la inferencia del modelo con los datos de testing
# en nuestro caso se uso con los datos de validacion por no contar con suficiente cantidad de datos(este proceso es algo forzado y erroneo)
inf = model.predict(x_val)

In [ ]:
#imprime los valores inferidos y los datos de testing original(en nuestro caso se uso los datos de validacion)
inf=inf.T
y_val =y_val.T
print("{0}-{1}".format(inf,y_val))

In [ ]:
# grafica los datos inferidos conjuntamente con los datos de testing(en nuestrp caso los datos de validacion)
n = 6
plt.plot(inf[n])
plt.plot(y_val[n])
plt.title('Inference')
plt.ylabel('Value')
plt.xlabel('Data')
plt.legend(['prediction', 'real_data'], loc='upper right')
plt.show()

In [ ]:
#clean GPU memory
def limit_mem():
    K.get_session().close()
    cfg = tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    tf.Session(config=cfg)
limit_mem()

# Transfer learning
Existe diferentes maneras de aplicar transferencia de aprendisaje a un modelo entrenado, en nuestro caso se entreno el modelo completo puesto que solo esta formado por una capa de LSTM, y no se aplican varios conceptos pertienentes a la transferencia de aprendisaje por razones de la limitada cantidad de informacion.

In [ ]:
#carga los pesos entrenados del modelo para su posterior uso de prueba
#observacion: si se se desea realizar transferencia de aprendisaje es necesario entrenar el modelo con un dataset diferente al original
path_file = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
WEIGHTS_PATH = os.path.join(path_file,'pesos/pesos.h5')

In [ ]:
def model(step_input=n_in,
              step_out=n_out,
              size_input=1,
              output_dim=1,
              encoder_units=50):
    '''
            step_input: longitud de la entrada hacia el lstm(datos pasados para predecir un valor futuro)
              step_out: alcanze de la inferencia, en nuestro caso se infiere un año,
              size_input: tamaño de la entrada de los datos,
              output_dim: tamaño de la salida de los datos,
              encoder_units: espacio dimencional de salida de la LSTM
    '''
    X = Input(shape = (step_input, size_input))
    #primera capa de LSTM
    lstm= LSTM(encoder_units, activation="tanh", return_sequences=False)(X)
    #capa fully conected con una neurona
    out = Dense(1)(lstm)
    model = Model(inputs = [X], outputs = out)
    ### END CODE HERE ###
        
    return model

In [ ]:
model = model()

In [ ]:
model.load_weights(WEIGHTS_PATH, by_name = True)

In [ ]:
# compile model(estos datos pueden cambiar no siempre es identico al entrenamiento, se recomienda disminuir la tasa de aprendisaje)
#se utiliza el optimizador root mean square con los siguientes hyperparametros
opt = RMSprop(lr=0.45, rho=0.94, epsilon=1.0, decay=0.1)
#se utiliza la tecnica de early stoping con el objetivo de evitar el sobre ajuste de la red, y deteniendo
#el proceso se detiene despues de 4 itereaciones si no existiera una mejora
early_stopping=EarlyStopping(monitor='val_loss', patience=4)
#se guarda los pesos del modelo como pesos.h5 si en cada paso de todo el conjunto de datos existiera una mejora
mcp_save = ModelCheckpoint('pesos.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')
#las metricas de entrenamiento y la perdida se configuran con mean square error
model.compile(optimizer=opt, metrics=["mae"], loss='mae')

In [ ]:
# train model
History=model.fit(x_train, y_train, epochs=1000,  callbacks=[early_stopping,mcp_save], batch_size=4, validation_data=(x_val,y_val))

In [ ]:
# Modulo de limpieza de memoria del GPU
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
limit_mem()